# Imports


In [25]:

import torch
import torchvision
import torch.nn as nn  
import torch.optim as optim  
import torch.nn.functional as F
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from torch.utils.data import (
    DataLoader,
)  
from torch.utils.tensorboard import SummaryWriter 

# Simple CNN Architecture

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x



# Set device 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Declaring Hyperparameters

In [ ]:
in_channels = 1
num_classes = 10
num_epochs = 3

# Load the Dataset

In [ ]:
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)


# To do Hyperparameter Search

In [ ]:
batch_sizes = [32, 256]
learning_rates = [1e-2, 1e-3, 1e-4, 1e-5]
classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

Training The Model

In [ ]:
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        
        step = 0
        model = CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device)
        model.train()

        criterion = nn.CrossEntropyLoss()
        train_loader = DataLoader(
            dataset=train_dataset, batch_size=batch_size, shuffle=True
        )

        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
        writer = SummaryWriter(
            f"runs/MNIST/MiniBatchSize {batch_size} LR {learning_rate}"
        )

        images, _ = next(iter(train_loader))
        writer.add_graph(model, images.to(device))
        writer.close()

        for epoch in range(num_epochs):
            losses = []
            accuracies = []

            for batch_idx, (data, targets) in enumerate(train_loader):
                
                data = data.to(device=device)
                targets = targets.to(device=device)

                # forward
                scores = model(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())

                # backward
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Calculate 'running' training accuracy
                features = data.reshape(data.shape[0], -1)
                img_grid = torchvision.utils.make_grid(data)
                _, predictions = scores.max(1)
                num_correct = (predictions == targets).sum()
                running_train_acc = float(num_correct) / float(data.shape[0])
                accuracies.append(running_train_acc)

                # For plotting things in tensorboard
                class_labels = [classes[label] for label in predictions]
                writer.add_image("mnist_images", img_grid)
                writer.add_histogram("fc1", model.fc1.weight)
                writer.add_scalar("Training loss", loss, global_step=step)
                writer.add_scalar(
                    "Training Accuracy", running_train_acc, global_step=step
                )

                if batch_idx == 230:
                    writer.add_embedding(
                        features,
                        metadata=class_labels,
                        label_img=data,
                        global_step=batch_idx,
                    )
                step += 1

            writer.add_hparams(
                {"lr": learning_rate, "bsize": batch_size},
                {
                    "accuracy": sum(accuracies) / len(accuracies),
                    "loss": sum(losses) / len(losses),
                },
            )

#Installing The Tensorboard if not Available

In [27]:
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Closing the SummaryWriter instance

In [28]:
writer.close()

#Load TensorBoard

In [30]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Start TensorBoard

In [31]:
!tensorboard --logdir runs/

2023-04-16 17:13:59.049457: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.2 at http://localhost:6006/ (Press CTRL+C to quit)
W0416 17:14:32.625130 14066381012761

#Open TensorBoard Data in TensorBoard.dev

In [33]:
!tensorboard dev upload --logdir runs \
--name "My latest experiment" \
--description "Simple comparison of several hyperparameters"

2023-04-16 17:20:54.359947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/bdtro500SxS93NiAAH4Pmg/

[2023-04-16T17:20:57] Started scanning logdir.
E0416 17:21:14.795885 140266088019776 uploader.py:1122] Attempted to re-upload existing blob.  Skipping.
E0416 17:21:23.763013 140266088019776 uploader.py:1122] Attempted to re-upload existing blob.  Skipping.
[2023-04-16T17:26:40] Total uploaded: 53520 scalars, 26820 tensors (171.4 MB), 8 binary objects (45.1 kB)
Total skipped: 2 binary objects (11.3 kB)


Interrupted. View your TensorBoard at https://tensorboard.dev/experiment/bdtro500SxS93NiAAH4Pmg/
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/pyth

In [34]:
!zip -r /content/runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/MNIST/ (stored 0%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/ (stored 0%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/1681665192.1897235/ (stored 0%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/1681665192.1897235/events.out.tfevents.1681665192.97c7e8309fb6.230.48 (deflated 27%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/events.out.tfevents.1681665113.97c7e8309fb6.230.45 (deflated 81%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/events.out.tfevents.1681665114.97c7e8309fb6.230.46 (deflated 11%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/00230/ (stored 0%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/00230/default/ (stored 0%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/00230/default/sprite.png (deflated 6%)
  adding: content/runs/MNIST/MiniBatchSize 256 LR 1e-05/00230/default/tensors.tsv (deflated 93%)
  adding: content/runs/MNIST/M

#This is for Running the TensorBoard Files in Local Machine

In [35]:
from google.colab import files

In [36]:
files.download('/content/runs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>